In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [ ]:
import pandas as pd
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

In [ ]:
# [pin]
file_path = "../data/data-hard.csv"
root_path = "../data/"


df = pd.read_csv(file_path)
df["prefix"] = "clsorg"
df = df.rename({"message": "input_text", "label": "target_text"}, axis=1)
df.sample(20)

,input_text,target_text,prefix
2595,Криптобиржа Binance вернула возможность оплаты...,228-4,clsorg
6231,📂 Как отчиталась АФК «Система» за IV квартал ...,100-3,clsorg
3281,Так зачем Тинькофф КХЛ? Хоккей абсолютно искре...,115-3,clsorg
452,"""⚠️🇷🇺#CHMF #дивиденд акционеры """"Северсталь""...",152-2,clsorg
4661,🇷🇺#ABIO После интервью Марии Воронцовой о раз...,266-4,clsorg
5204,🇷🇺#TATN #дивиденды КОНСЕНСУС: Татнефть может ...,163-4,clsorg
3540,​Ослабление рубля. Как защититься? 🔶Диапазон ...,160-4;53-4,clsorg
3990,​​🟢 ИТОГИ ДНЯ. Российский рынок акций продолжи...,22-4,clsorg
843,"""🇷🇺#SNGS #дивиденд Дивдоходность """"префов"""" Су...",160-4,clsorg
2798,Ожидаемые события на 15 марта 🇷🇺 В России По...,225-3,clsorg


In [ ]:
m_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

In [ ]:
from t5src.dataset import NERDataModel
from t5src.model import NERModel

BATCH_SIZE = 64
EPOCHS = 10
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)
data_module = NERDataModel(train_df, test_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

In [ ]:
model = NERModel()

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="ner",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min",
)

trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=EPOCHS,
    accelerator="cuda",
)

In [ ]:
!rm -r lightning_logs

In [ ]:
trainer.fit(model, data_module)

In [ ]:
trained_model = NERModel.load_from_checkpoint("checkpoints/ner-v5.ckpt")
trained_model.freeze()

In [ ]:
from t5src.utils import evaluate_metric, generate_answer_batched

predictions = generate_answer_batched(
    trained_model=trained_model, tokenizer=tokenizer, data=test_df, batch_size=512
)

In [ ]:
ldf = test_df.copy()
ldf["predictions"] = predictions
ldf[["tcomp", "tsent"]] = (
    ldf["target_text"].str.split(";", expand=True)[0].str.split("-", expand=True)
)
ldf[["pcomp", "psent"]] = (
    ldf["predictions"].str.split(";", expand=True)[0].str.split("-", expand=True)
)

In [ ]:
# [pin]

evaluate_metric(
    company_predictions=ldf["pcomp"].tolist(),
    company_labels=ldf["tcomp"].tolist(),
    sentiment_predictions=ldf["psent"].tolist(),
    sentiment_labels=ldf["tsent"].tolist(),
)

{'total': 57.80824034396828,
 'f1': 0.6391920745477017,
 'accuracy': 0.5169727323316639}